# АВС - анализ


# 🎈🍾 🎉





Вы работаете в крупном онлайн-магазине, который реализует товары «всё для праздников». Вам необходимо провести ABC-анализ и выявить наиболее важные товары на основе объемов продаж и популярности. Эта информация составит план закупок на весь следующий год. 

**ABC-анализ** позволяет определить наиболее важные ресурсы компании на основе объемов продаж и прибыли.

В основе ABC-анализа лежит __«Принцип Парето»: 20% ресурсов приносят 80% прибыли__. Именно этот вид анализа позволяет выделить те самые  группы товаров, ресурсов, клиентов, поставщиков и т. п., которые и приносят компании основную прибыль.

__Задача анализа разделить товары на группы по принципу:__

- А — наиболее ценные, 20% — ассортимента (номенклатура); 80% — продаж
- В — промежуточные, 30% — ассортимента; 15% — продаж
- С — наименее ценные, 50% — ассортимента; 5% — продаж

## Задание 1

Сохраните все данные из таблицы `products` из PostgreSQL в дата-фрейм `products` и подготовьте данные к анализу.

1. Подгрузите данные для подключения
2. Подключите нужную библиотеку для работы с PostgreSQL
3. Создайте соединение с БД
4. Создайте запрос и выгрузите все данные из таблицы `products`
5. Положите данные в дата-фрейм `products`
6. Приведите данные к удобному типу: числа к числовому типу, идентификаторы — к текстовому


In [1]:
from postgresql_secret1 import pg_secret

In [2]:
import psycopg2

connection = psycopg2.connect(
                             database=pg_secret['db_name'],
                             user=pg_secret['db_user'],
                             password=pg_secret['db_password'],
                             host=pg_secret['db_host'],
                             port=pg_secret['db_port'],
                             )# создай соединение

cursor = connection.cursor()# заведи курсор, чтобы делать запросы в базу

In [3]:
import pandas as pd

query = """SELECT * 
           FROM products"""# напиши запрос, твоих знаний SQL точно хватит

cursor.execute(query)

result = cursor.fetchall()

result


[(524025, '21232', 'STRAWBERRY CERAMIC TRINKET BOX', 72, Decimal('1.06')),
 (524025, '85099C', 'JUMBO  BAG BAROQUE BLACK WHITE', 100, Decimal('1.65')),
 (524025, '21231', 'SWEETHEART CERAMIC TRINKET BOX', 72, Decimal('1.06')),
 (524025, '47570B', 'TEA TIME TABLE CLOTH', 50, Decimal('8.49')),
 (524025, '20685', 'DOORMAT RED RETROSPOT', 10, Decimal('6.75')),
 (524025, '20718', 'RED SPOTTY SHOPPER BAG', 100, Decimal('1.06')),
 (524025, '85150', 'LADIES & GENTLEMEN METAL SIGN', 48, Decimal('2.10')),
 (524025, '22804', 'CANDLEHOLDER PINK HANGING HEART', 64, Decimal('2.55')),
 (524026, '47559B', 'TEA TIME OVEN GLOVE', 100, Decimal('1.06')),
 (524026, '21154', 'RED RETROSPOT OVEN GLOVE ', 200, Decimal('1.06')),
 (524026, '47599A', 'PINK PARTY BAGS', 100, Decimal('1.69')),
 (524027, '22952', '72 CAKE CASES VINTAGE CHRISTMAS', 672, Decimal('0.42')),
 (524028, '85049E', 'SCANDINAVIAN REDS RIBBONS', 1, Decimal('1.25')),
 (524028, '22082', 'RIBBON REEL STRIPES DESIGN ', 1, Decimal('1.65')),
 (5240

In [4]:
cursor.description

(Column(name='invoice_id', type_code=23),
 Column(name='stock_code', type_code=1043),
 Column(name='description', type_code=1043),
 Column(name='quantity', type_code=23),
 Column(name='price', type_code=1700))

In [5]:
columns = []                     # В этот список будем сохранять названия колонок
for desc in cursor.description:  
    columns.append(desc[0])      # Названия колонок возьмем из первого элемента cursor.description

    
products = pd.DataFrame(result, columns=columns)  # Из списка кортежей сделаем датафрейм
products.head()

,invoice_id,stock_code,description,quantity,price
0,524025,21232,STRAWBERRY CERAMIC TRINKET BOX,72,1.06
1,524025,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,1.65
2,524025,21231,SWEETHEART CERAMIC TRINKET BOX,72,1.06
3,524025,47570B,TEA TIME TABLE CLOTH,50,8.49
4,524025,20685,DOORMAT RED RETROSPOT,10,6.75


In [6]:
products['invoice_id'] = products['invoice_id'].apply(str)
products['stock_code'] = products['stock_code'].apply(str)
products['description'] = products['description'].apply(str)
products['quantity'] = products['quantity'].apply(int)
products['price'] = products['price'].apply(float)

## Задание 2

__Посчитайте, какой доход принес каждый продукт и сколько единиц каждого продукта было продано__

1. Посчитай общую сумму оплат для каждой строки счета
2. Получи общий доход и количество единиц по товарам
3. Объедини сгруппированные данные в один датафрейм со столбцами: `description`, `stock_code`, `quantity`, `total`

In [7]:
products['total'] = products['quantity'] * products['price']
products.head()

,invoice_id,stock_code,description,quantity,price,total
0,524025,21232,STRAWBERRY CERAMIC TRINKET BOX,72,1.06,76.32
1,524025,85099C,JUMBO BAG BAROQUE BLACK WHITE,100,1.65,165.00
2,524025,21231,SWEETHEART CERAMIC TRINKET BOX,72,1.06,76.32
3,524025,47570B,TEA TIME TABLE CLOTH,50,8.49,424.50
4,524025,20685,DOORMAT RED RETROSPOT,10,6.75,67.50


In [8]:
# посчитай общих доход по каждому товару
total_stock = products.groupby('stock_code')['total'].sum().reset_index()
total_stock.head()

,stock_code,total
0,10002,6638.27
1,10080,124.61
2,10109,1.68
3,10120,136.08
4,10123C,226.76


In [9]:
# посчитай общее количество проданных единиц по каждому товару
quantity_stock = products.groupby('stock_code')['quantity'].sum().reset_index()
quantity_stock.head()

,stock_code,quantity
0,10002,8479
1,10080,303
2,10109,4
3,10120,648
4,10123C,628


In [10]:
# объедини общие данные в одну таблицу
qua_tot = pd.merge(total_stock, quantity_stock, on='stock_code')
qua_tot.head()

,stock_code,total,quantity
0,10002,6638.27,8479
1,10080,124.61,303
2,10109,1.68,4
3,10120,136.08,648
4,10123C,226.76,628


## Задание 3

__Найди нарастающий итог по процентам__

1. Отсортируй датафрейм по доходу в нисходящем порядке 
2. Найди процент дохода каждого товара в общем доходе 
3. Сделай нарастающий итог (running total) по процентам каждого товара, для этого найди в интернете и примени метод для расчета кумулятивной суммы в `pandas`. 


Нарастающий итог — это представление данных, в котором мы видим динамику нарастания или убывания суммы от начала отсчета до конца. Получаем мы эту кумулятивную сумму, прибавлением поочередно каждого следующего элемента к сумме предыдущих.  



In [11]:
# отсортируй дата-фрейм (большие вверху, маленькие внизу)
sorted_qua_tot = qua_tot.sort_values(by='total' , ascending=False)
sorted_qua_tot.head()

,stock_code,total,quantity
1609,22423,277656.25,24139
4080,85123A,247203.36,91814
2726,23843,168469.60,80995
4058,85099B,167920.64,93436
4625,M,151777.67,9391


In [12]:
# найди процент дохода для каждого товара 

#(искомая часть / целое число) * 100

sorted_qua_tot['per_revenue'] = sorted_qua_tot['total'] / sorted_qua_tot['total'].sum() *100
sorted_qua_tot

,stock_code,total,quantity,per_revenue
1609,22423,277656.25,24139,1.598040
4080,85123A,247203.36,91814,1.422769
2726,23843,168469.60,80995,0.969620
4058,85099B,167920.64,93436,0.966461
4625,M,151777.67,9391,0.873550
...,...,...,...,...
4630,TEST002,1.00,1,0.000006
247,20721,0.85,1,0.000005
3419,84205C,0.38,2,0.000002
2847,35930,0.38,1,0.000002


In [13]:
# используй метод pandas для кумулятивной суммы
running_total = pd.DataFrame(sorted_qua_tot.groupby('stock_code')['per_revenue'].sum()).sort_values(by='per_revenue' , ascending=False).reset_index()
running_total['cum_sum'] = running_total['per_revenue'].cumsum()

running_total

,stock_code,per_revenue,cum_sum
0,22423,1.598040,1.598040
1,85123A,1.422769,3.020809
2,23843,0.969620,3.990429
3,85099B,0.966461,4.956890
4,M,0.873550,5.830440
...,...,...,...
4626,TEST002,0.000006,99.999991
4627,20721,0.000005,99.999996
4628,84205C,0.000002,99.999998
4629,35930,0.000002,100.000000


## Задание 4

__Пометь товары метками `'A'`, `'B'`, `'C'` в зависимости от нарастающего итога__

1. Напиши функцию, которая будет помечать товары с нарастающим итогом ниже или равно 80% как группу `'A'`, от 80%-95% — как `'B'` и остальные как `'C'`

In [16]:
def revenue(x):

    if x <= 80:
        return 'A'
    elif x > 80 and x <= 95:
        return 'B'
    else:
        return 'C'    # допиши функцию

# создай новый столбец revenue с помощью функции
running_total['revenue'] = running_total['cum_sum'].apply(revenue) 
running_total

,stock_code,per_revenue,cum_sum,revenue
0,22423,1.598040,1.598040,A
1,85123A,1.422769,3.020809,A
2,23843,0.969620,3.990429,A
3,85099B,0.966461,4.956890,A
4,M,0.873550,5.830440,A
...,...,...,...,...
4626,TEST002,0.000006,99.999991,C
4627,20721,0.000005,99.999996,C
4628,84205C,0.000002,99.999998,C
4629,35930,0.000002,100.000000,C


## Задание 5

__Найди нарастающий итог по количеству товарных единиц и пометь товары метками `'A'`, `'B'`, `'C'`, отмечая тем самым популярность товара__

1. Отсортируй предыдущий датафрейм с данными о доходе теперь по количеству товаров
2. Найди процент каждого товара в общем количестве
3. Найди нарастающий итог по количеству
4. Напиши функцию с теми же параметрами, что и у дохода
5. Отметь группу

In [18]:
sorted_qua_tot = qua_tot.sort_values(by='quantity' , ascending=False)
sorted_qua_tot.head()

,stock_code,total,quantity
3410,84077,24098.03,105185
4058,85099B,167920.64,93436
4080,85123A,247203.36,91814
601,21212,43390.03,89850
2726,23843,168469.60,80995


In [19]:
# проведи те же операции, что в двух предыдущих заданиях, для оценки популярности товаров
sorted_qua_tot['per_quantity'] = sorted_qua_tot['quantity'] / sorted_qua_tot['quantity'].sum() *100
sorted_qua_tot

,stock_code,total,quantity,per_quantity
3410,84077,24098.03,105185,0.999031
4058,85099B,167920.64,93436,0.887441
4080,85123A,247203.36,91814,0.872035
601,21212,43390.03,89850,0.853381
2726,23843,168469.60,80995,0.769278
...,...,...,...,...
2936,37484,6.75,1,0.000009
4144,85171,2.55,1,0.000009
2917,37451,4.95,1,0.000009
2907,37438,3.75,1,0.000009


In [21]:
running_quantity = pd.DataFrame(sorted_qua_tot.groupby('stock_code')['per_quantity'].sum()).sort_values(by='per_quantity' , ascending=False).reset_index()
running_quantity['cum_sum_qua'] = running_quantity['per_quantity'].cumsum()

running_quantity

,stock_code,per_quantity,cum_sum_qua
0,84077,0.999031,0.999031
1,85099B,0.887441,1.886471
2,85123A,0.872035,2.758506
3,21212,0.853381,3.611888
4,23843,0.769278,4.381166
...,...,...,...
4626,37484,0.000009,99.999962
4627,85171,0.000009,99.999972
4628,37451,0.000009,99.999981
4629,37438,0.000009,99.999991


In [29]:
def quantity(x):

    if x <= 80:
        return 'A'
    elif x > 80 and x <= 95:
        return 'B'
    else:
        return 'C'   # допиши функцию

# создай новый столбец popularity с помощью функции
running_quantity['popularity'] = running_quantity['cum_sum_qua'].apply(quantity) 
running_quantity

,stock_code,per_quantity,cum_sum_qua,popularity
0,84077,0.999031,0.999031,A
1,85099B,0.887441,1.886471,A
2,85123A,0.872035,2.758506,A
3,21212,0.853381,3.611888,A
4,23843,0.769278,4.381166,A
...,...,...,...,...
4626,37484,0.000009,99.999962,C
4627,85171,0.000009,99.999972,C
4628,37451,0.000009,99.999981,C
4629,37438,0.000009,99.999991,C


In [30]:
prod = pd.merge(running_total,running_quantity,on='stock_code').reset_index()
prod = prod[['stock_code','revenue','popularity']]
prod

,stock_code,revenue,popularity
0,22423,A,A
1,85123A,A,A
2,23843,A,A
3,85099B,A,A
4,M,A,A
...,...,...,...
4626,TEST002,C,C
4627,20721,C,C
4628,84205C,C,C
4629,35930,C,C


## Задание 6

__Объедини группы и проанализируй__

1. Создай серию `group` с объединенным столбцом отметок популярности и доходности товаров
2. Посчитай количество вхождений каждой сгруппированной пары
3. Какие группы непопулярны и приносят мало дохода?
4. На каких группах товаров нужно сфокусироваться при составлении плана закупок?

In [31]:
# напиши здесь код и комментарии
prod['group']=prod['revenue'] + prod['popularity']

prod.head()

,stock_code,revenue,popularity,group
0,22423,A,A,AA
1,85123A,A,A,AA
2,23843,A,A,AA
3,85099B,A,A,AA
4,M,A,A,AA


In [32]:
prod['group'].value_counts()

CC    2041
AA     685
BB     577
BC     377
CB     303
BA     270
AB     248
CA      72
AC      58
Name: group, dtype: int64

Hепопулярны и приносят мало дохода группы BC, CB и CC.
При составлении плана закупок нужно сфокусироваться на группах товаров AA, AB и BA.

## Задание 7

__Выгрузи данные по товарам с метокой по ABC-анализу__

1. Объедини по `stock_code` в одной таблице `ABC-код`, описание товара и `ID `

2. Выгрузи таблицу в `csv` для отправки в отдел закупок

In [46]:
prodABC = pd.merge(products, prod, on='stock_code' , how='inner')
#prodABC = [['stock_code', 'group','description','invoice_id']]
prodABC

,invoice_id,stock_code,description,quantity,price,total,revenue,popularity,group
0,524025,21232,STRAWBERRY CERAMIC TRINKET BOX,72,1.06,76.32,A,A,AA
1,524091,21232,STRAWBERRY CERAMIC TRINKET BOX,12,1.25,15.00,A,A,AA
2,524099,21232,STRAWBERRY CERAMIC TRINKET BOX,12,1.25,15.00,A,A,AA
3,524184,21232,STRAWBERRY CERAMIC TRINKET BOX,3,1.25,3.75,A,A,AA
4,524206,21232,STRAWBERRY CERAMIC TRINKET BOX,12,1.25,15.00,A,A,AA
...,...,...,...,...,...,...,...,...,...
779490,513566,ADJUST2,Adjustment by Peter on Jun 25 2010,1,358.47,358.47,C,C,CC
779491,513565,ADJUST2,Adjustment by Peter on Jun 25 2010,1,72.45,72.45,C,C,CC
779492,515916,45014,FOLDING SHIRT TIDY,1,4.65,4.65,C,C,CC
779493,517719,21585,SWALLOW SMALL TUBE MATCHES,1,1.65,1.65,C,C,CC


In [48]:
prod_ABC = prodABC [['stock_code', 'group','description','invoice_id']]
prod_ABC.head()

,stock_code,group,description,invoice_id
0,21232,AA,STRAWBERRY CERAMIC TRINKET BOX,524025
1,21232,AA,STRAWBERRY CERAMIC TRINKET BOX,524091
2,21232,AA,STRAWBERRY CERAMIC TRINKET BOX,524099
3,21232,AA,STRAWBERRY CERAMIC TRINKET BOX,524184
4,21232,AA,STRAWBERRY CERAMIC TRINKET BOX,524206


In [49]:
prod_ABC.to_csv('prod_ABC.csv', index=False)

# 🎉🎉🎉 Поздравляем, ты великолепен! 